# ZenML Pipelines for SKLearn

In [ ]:
!pip install zenml
!zenml integration install sklearn

## Initialize the Project

In [1]:
!zenml connect --url http://192.168.2.110:8888 --username admin --password zenml

NumExpr defaulting to 8 threads.
Connecting to: 'http://192.168.2.110:8888'...
Updated the global store configuration.


In [ ]:
!rm -rf .zen
!zenml init

## Build an SciKit-Learn SVC Image Classifier

In [3]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [17]:
def train_test():
    """Train and test a SKLearn SVC classifier on digits"""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )

    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    print(f"Test Accuracy: {score}")

In [18]:
train_test()

Test Accuracy: 0.9583333333333334


## Run the Classifier using a ZenML Pipeline

### Define Steps

Define the classifier in 3 ZenML Pipeline steps - __Data Loading__, __Model Training__ and __Model Evaluation__.

In [4]:
from zenml import step
from typing_extensions import Annotated
from typing import Tuple

In [5]:
@step
def importer() -> Tuple[
    Annotated[np.ndarray, "X_train"],
    Annotated[np.ndarray, "X_test"],
    Annotated[np.ndarray, "y_train"],
    Annotated[np.ndarray, "y_test"],
]:
    """Load the digits dataset as numpy arrays."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train, X_test, y_train, y_test = train_test_split(
        data, digits.target, test_size=0.2, shuffle=False
    )
    return X_train, X_test, y_train, y_test

NumExpr defaulting to 8 threads.


In [6]:
@step
def svc_trainer(
            X_train: np.ndarray,
            y_train: np.ndarray
        ) -> ClassifierMixin:
    """Train the SVC classifier."""
    model = SVC(gamma=0.001)
    model.fit(X_train, y_train)

    return model

In [7]:
@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin
) -> float:
    """Calculate the model accuracy using the test set."""
    score = model.score(X_test, y_test)
    print(f"Test Accuracy: {score}")

    return score

### Define Pipeline of Steps

In [8]:
from zenml import pipeline

In [9]:
@pipeline
def digits_classifier():
    """SVC digits classifier pipeline"""
    X_train, X_test, y_train, y_test = importer()
    model = svc_trainer(X_train, y_train=y_train)
    evaluator(X_test=X_test, y_test=y_test, model=model)

### Initialize Pipeline

In [10]:
classifier = digits_classifier()

Initiating a new run for the pipeline: digits_classifier.
Registered new version: (version 1).
Executing a new run.
Using user: admin
Using stack: default
  orchestrator: default
  artifact_store: default
Step importer has started.
Step importer has finished in 3.163s.
Step svc_trainer has started.
Step svc_trainer has finished in 0.644s.
Step evaluator has started.
Test Accuracy: 0.9583333333333334
Step evaluator has finished in 0.582s.
Run digits_classifier-2023_09_27-08_22_04_011726 has finished in 6.684s.
Dashboard URL: http://192.168.2.110:8888/workspaces/default/pipelines/32b2fbc0-765b-48d9-badd-75d60e1f46fa/runs/a9ffc4c6-5a2b-4560-bb8b-beab259b4bf1/dag


## Visualize the Results

Head over to the dashboard URL given above to see the visualization of the pipeline run:

![ZenML Pipelines for SKLearn](../assets/ZenML_MLFlow_01.webp)